<a href="https://colab.research.google.com/github/Talendar/process_scheduling_simulation/blob/main/process_scheduling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Process Scheduling Algorithms

In [45]:
from copy import deepcopy
CTX_SW_COST = 1   # context switching cost (in time units)


class Process: 
  def __init__(self, name, time, priority=1):
    self.name = name
    self.time = time
    self.priority = priority
    self.turnaround_time = -1

  def __str__(self):
    return "< Name: %s  |  Time: %d  |  Priority: %d  |  Turnaround time: %d >" % (
        self.name, self.time, self.priority, self.turnaround_time)


def print_results(results, alg_name):
  print("< %s >\nTotal time: %ds \nTurnaround times:" % (alg_name, results[0]))
  for p in results[1]:
    print("   [%s] %ds" % (p.name, p.turnaround_time))

In [48]:
# process list for exercise 16
processes_ex16 = [
      Process("A", 10), 
      Process("B", 6), 
      Process("C", 2), 
      Process("D", 4), 
      Process("E", 8),
]

# process list for exercise 33
processes_ex33 = [
      Process("A", 10, 5), 
      Process("B", 11, 5), 
      Process("C", 5, 1), 
      Process("D", 15, 2), 
      Process("E", 10, 2), 
      Process("F", 12, 3), 
      Process("G", 9, 4),
]

### Round-Robin algorithm

In [4]:
def round_robin(proc_list, quantum):
  finished = []
  dT = 0
  while proc_list:
    for p in proc_list:
      dT += CTX_SW_COST
      if p.time <= quantum:
        dT += p.time
        p.time = 0
        p.turnaround_time = dT
        proc_list.remove(p)
        finished.append(p)
      else:
        p.time -= quantum
        dT += quantum
  return dT, finished

In [26]:
# exercise 16
res_rr16 = round_robin(deepcopy(processes_ex16), quantum=3)
print(">>  Ex. 16  <<\n")
print_results(res_rr16, "Round-robin")

>>  Ex. 16  <<

< Round-robin >
Total time: 42s 
Turnaround times:
   [C] 11s
   [B] 23s
   [E] 38s
   [A] 40s
   [D] 42s


In [25]:
# exercise 33
res_rr33 = round_robin(deepcopy(processes_ex33), quantum=4)
print(">>  Ex. 33  <<\n")
print_results(res_rr33, "Round-robin")

>>  Ex. 33  <<

< Round-robin >
Total time: 93s 
Turnaround times:
   [C] 47s
   [A] 65s
   [E] 73s
   [G] 75s
   [B] 79s
   [F] 84s
   [D] 93s


### Shortest Job First algorithm

In [14]:
def sjf(proc_list):
  finished = []
  dT = 0
  while proc_list:
    dT += CTX_SW_COST
    p = min(proc_list, key=(lambda x: x.time))
    dT += p.time 
    p.time = 0
    p.turnaround_time = dT
    proc_list.remove(p)
    finished.append(p)
  return dT, finished

In [24]:
# exercise 16
res_sjf16 = sjf(deepcopy(processes_ex16))
print(">>  Ex. 16  <<\n")
print_results(res_sjf16, "SJF")

>>  Ex. 16  <<

< SJF >
Total time: 35s 
Turnaround times:
   [C] 3s
   [D] 8s
   [B] 15s
   [E] 24s
   [A] 35s


### First-in First-out algorithm

In [16]:
def fifo(proc_list):
  finished = []
  dT = 0
  while proc_list:
    dT += CTX_SW_COST
    p = proc_list.pop(0)
    dT += p.time
    p.time = 0
    p.turnaround_time = dT
    finished.append(p)
  return dT, finished

In [23]:
# exercise 16
res_fifo16 = fifo(deepcopy(processes_ex16))
print(">>  Ex. 16  <<\n")
print_results(res_fifo16, "FIFO")

>>  Ex. 16  <<

< FIFO >
Total time: 35s 
Turnaround times:
   [A] 11s
   [B] 18s
   [C] 21s
   [D] 26s
   [E] 35s


In [22]:
# exercise 33
res_fifo33 = fifo(deepcopy(processes_ex33))
print(">>  Ex. 33  <<\n")
print_results(res_fifo33, "FIFO")

>>  Ex. 33  <<

< FIFO >
Total time: 79s 
Turnaround times:
   [A] 11s
   [B] 23s
   [C] 29s
   [D] 45s
   [E] 56s
   [F] 69s
   [G] 79s


### Priority Scheduling algorithm

In [74]:
def priority_scheduling(proc_list, priorities_quanta):
  proc_priority_lists = {key: [] for key in priorities_quanta}
  for p in proc_list:
    proc_priority_lists[p.priority].append(p)

  finished = []
  dT = 0
  for priority, p_list in sorted(proc_priority_lists.items(), reverse=True):
      t, f = round_robin(p_list, quantum=priorities_quanta[priority])
      for p in f:
        p.turnaround_time += dT
        finished.append(p)
      dT += t
  return dT, finished

In [75]:
# exercise 33
res_priority33 = priority_scheduling(
    proc_list=deepcopy(processes_ex33),
    priorities_quanta={
        1: 4, 2: 4, 3: 4,
        4: 5, 5: 5,                                  
    })
print(">>  Ex. 33  <<\n")
print_results(res_priority33, "Priority Scheduling")

>>  Ex. 33  <<

< Priority Scheduling >
Total time: 91s 
Turnaround times:
   [A] 18s
   [B] 26s
   [G] 37s
   [F] 52s
   [E] 80s
   [D] 84s
   [C] 91s
